Dog vs Cat Classification using Pretrained ResNet-101

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import warnings

In [ ]:
from PIL import Image, UnidentifiedImageError
warnings.filterwarnings("ignore", category=UserWarning)


1. Device Configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Transforming the images

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

 Safe Loader used:(skipping the corrupted images)

In [ ]:
def safe_loader(path):
    try:
        with open(path, "rb") as f:
            img = Image.open(f).convert("RGB")
        return img
    except (UnidentifiedImageError, OSError):
        # Return a blank image if corrupted
        return Image.new("RGB", (224, 224))

Dataset & DataLoader

In [ ]:
data_dir = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages"

full_dataset = datasets.ImageFolder(root=data_dir,
                                    transform=data_transform,
                                    loader=safe_loader)

train_size = int(0.8 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}")

Load Pretrained ResNet-101

In [ ]:
model = models.resnet101(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

Training the Model

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Acc: {train_acc:.2f}%")

Evaluating the Model

In [ ]:
# =====================================
# 7. Validation Loop
# =====================================
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")

Visualize Predictions

In [ ]:
# =====================================
# 8. Quick Prediction & Visualization
# =====================================
import matplotlib.pyplot as plt
import torchvision
import numpy as np

# Class names
classes = ['Cat', 'Dog']

# Simple imshow (unnormalize roughly)
def imshow(img):
    img = img / 2 + 0.5  # unnormalize (approx)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()

# Get a batch of validation images
dataiter = iter(val_loader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)

# Run through model
outputs = model(images[:4])   # <-- using our ResNet101 model
_, predicted = torch.max(outputs, 1)

# Show images
imshow(torchvision.utils.make_grid(images[:4].cpu()))

# Print Ground Truth vs Predicted
print('GroundTruth: ', ' '.join(classes[labels[j]] for j in range(4)))
print('Predicted:   ', ' '.join(classes[predicted[j]] for j in range(4)))